## Human Activity Recognition using RNN

### 0. Install the dependacy packages

In [1]:
!pip install numpy pandas tensorflow

### 1. Load the training and test data

#### 1.1 make the necessary imports

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout, LSTM
from tensorflow.keras.utils import to_categorical


2022-11-28 08:29:25.872906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### 1.2 Uitility functions for loading the data

In [3]:
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [4]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

In [5]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [6]:
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'dataset/')
    return trainX, trainy, testX, testy


In [7]:
train_X, train_Y, test_X, test_Y = load_dataset()

### 2. Try to understand the data

In [8]:
print(train_X.shape)
print(train_Y.shape)

(7352, 128, 9)
(7352, 1)


In [9]:
print(test_X.shape)
print(test_Y.shape)

(2947, 128, 9)
(2947, 1)


In [12]:
# Features for a single subject in a single timestampe
train_X[1,1,:]

array([ 1.02238   , -0.1268078 ,  0.09935086,  0.00455008, -0.00748789,
       -0.02509841,  0.02417851,  0.00971036,  0.01614958])

In [13]:
# Possible output values
np.unique(train_Y)

array([1, 2, 3, 4, 5, 6])

### 3. Preprocessing

#### 2.1 Zero offset the output class values and one-hot encode

The output data set is in the values range 1-6. We'll convert this to the range 0-5 and one hot encode it

In [14]:
# zero-offset class values
train_Y,test_Y = train_Y -1, test_Y - 1
# one hot encode y
train_Y = to_categorical(train_Y)
test_Y = to_categorical(test_Y)

In [20]:
print(train_X.shape)
print(train_Y.shape)

(7352, 128, 9)
(7352, 6)


### 4. Train

#### 4.1 Prepare the model

In [22]:
n_timesteps, n_features, n_outputs= train_X.shape[1], train_X.shape[2], train_Y.shape[1]

In [29]:
model = Sequential()
model.add(LSTM(100, input_shape = (n_timesteps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 4.2 Fit the model

In [30]:
epochs = 15
batch_size = 64
model.fit(train_X, train_Y, epochs = epochs, batch_size=batch_size)

Epoch 1/15
115/115 [==============================] - 7s 46ms/step - loss: 1.1267 - accuracy: 0.5249
Epoch 2/15
115/115 [==============================] - 5s 47ms/step - loss: 0.6757 - accuracy: 0.7089
Epoch 3/15
115/115 [==============================] - 5s 46ms/step - loss: 0.5106 - accuracy: 0.7816
Epoch 4/15
115/115 [==============================] - 5s 46ms/step - loss: 0.3873 - accuracy: 0.8502
Epoch 5/15
115/115 [==============================] - 5s 46ms/step - loss: 0.2912 - accuracy: 0.9029
Epoch 6/15
115/115 [==============================] - 5s 46ms/step - loss: 0.1827 - accuracy: 0.9314
Epoch 7/15
115/115 [==============================] - 5s 47ms/step - loss: 0.1669 - accuracy: 0.9363
Epoch 8/15
115/115 [==============================] - 6s 48ms/step - loss: 0.1519 - accuracy: 0.9400
Epoch 9/15
115/115 [==============================] - 6s 48ms/step - loss: 0.2270 - accuracy: 0.9185
Epoch 10/15
115/115 [==============================] - 6s 49ms/step - loss: 0.1913 - accura

### 5. Evaluate

In [31]:
results = model.evaluate(test_X, test_Y, batch_size = batch_size, verbose=0)

In [28]:
print("test loss, test acc:", results)

test loss, test acc: [0.3805740177631378, 0.9117746949195862]


In [32]:
print("test loss, test acc:", results)

test loss, test acc: [0.23025718331336975, 0.9073634147644043]
